## No vector computing

In [1]:
import compute_std
import function_set_std

In [ ]:
alias_values = {'1AJ': 30000.}

In [ ]:
input_values = compute_std.prepare(alias_values)

In [ ]:
compute_std.compute(input_values)

In [ ]:
%timeit compute_std.compute(input_values)

## Vector computing

In [2]:
import compute_np
import function_set_np

In [3]:
alias_values = {'1AJ': 30000.}

In [4]:
input_values = compute_np.prepare(alias_values)

In [5]:
compute_np.compute(input_values)

array([ 2461.,  2461.,  2461., ...,  2461.,  2461.,  2461.])

In [6]:
%timeit compute_np.compute(input_values)

1 loop, best of 3: 4.42 s per loop
